# Letters Recognition - Part 2

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In the Part 1 we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

In [2]:
pickle_file = 'notMNIST.pickle'

#to reload the data
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train: data as a flat matrix,
labels as float 1-hot encodings.

In [3]:
image_size = 28 # we have 28 by 28 pictures
num_labels = 10 # we have 10 different letters

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We start with gradient descent training. We will use only a 
subset the training data for faster turnaround.

In [4]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.811075
Training accuracy: 10.6%
Validation accuracy: 13.9%
Loss at step 100: 2.245941
Training accuracy: 72.5%
Validation accuracy: 70.8%
Loss at step 200: 1.792173
Training accuracy: 75.5%
Validation accuracy: 72.8%
Loss at step 300: 1.554715
Training accuracy: 76.6%
Validation accuracy: 73.5%
Loss at step 400: 1.396363
Training accuracy: 77.7%
Validation accuracy: 73.9%
Loss at step 500: 1.279856
Training accuracy: 78.4%
Validation accuracy: 73.9%
Loss at step 600: 1.188971
Training accuracy: 78.9%
Validation accuracy: 74.1%
Loss at step 700: 1.115077
Training accuracy: 79.2%
Validation accuracy: 74.2%
Loss at step 800: 1.053303
Training accuracy: 79.7%
Validation accuracy: 74.5%
Test accuracy: 82.5%


Now we will try stochastic gradient descent training instead, which is much faster.
The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.698720
Minibatch accuracy: 10.2%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.108249
Minibatch accuracy: 78.9%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 1.416429
Minibatch accuracy: 78.1%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 0.872192
Minibatch accuracy: 75.8%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 0.951792
Minibatch accuracy: 83.6%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.877005
Minibatch accuracy: 76.6%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.967088
Minibatch accuracy: 78.1%
Validation accuracy: 77.8%
Test accuracy: 86.2%


We got a better accuracy :)

Now I am going to turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu(). This model should improve test accuracy.

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
	# Input data. For the training data, we use a placeholder that will be fed
	# at run time with a training minibatch.
	tf_train_dataset = tf.placeholder(tf.float32,
										shape=(batch_size, image_size * image_size))
	tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
	tf_valid_dataset = tf.constant(valid_dataset)
	tf_test_dataset = tf.constant(test_dataset)

	# Variables.
	num_hidden_nodes = 1554
	weights1 = tf.Variable(
		tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
	weights2 = tf.Variable(
		tf.truncated_normal([num_hidden_nodes, num_labels]))
	biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
	biases2 = tf.Variable(tf.zeros([num_labels]))

	# Training computation.
	layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
	logits = tf.matmul(layer_1, weights2) + biases2
	loss = tf.reduce_mean(
		tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

	# Optimizer.
	optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

	# Predictions for the training, validation, and test data.
	train_prediction = tf.nn.softmax(logits)
	valid_prediction = tf.nn.softmax(
		tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1),weights2) + biases2)
	test_prediction = tf.nn.softmax(
		tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1),weights2) + biases2)


In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
	tf.initialize_all_variables().run()
	print("Two layers netowrk, Initialized")
	for step in range(num_steps):
		offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
		batch_data = train_dataset[offset:(offset + batch_size), :]
		batch_labels = train_labels[offset:(offset + batch_size), :]
		feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
		_, l, predictions = session.run(
			[optimizer, loss, train_prediction], feed_dict=feed_dict)
		if (step % 500 == 0):
			print("Minibatch loss at step %d: %f" % (step, l))
			print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
			print("Validation accuracy: %.1f%%" % accuracy(
				valid_prediction.eval(), valid_labels))
	print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Two layers netowrk, Initialized
Minibatch loss at step 0: 428.107574
Minibatch accuracy: 8.6%
Validation accuracy: 26.8%
Minibatch loss at step 500: 29.696074
Minibatch accuracy: 77.3%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 16.196051
Minibatch accuracy: 78.1%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 7.194921
Minibatch accuracy: 88.3%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 7.381135
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 4.869624
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 3.532003
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Test accuracy: 89.8%


Now we got 90% accuracy!

## L2 Regularizaton for the logistic model. 

In [20]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(tf_train_dataset, weights) + biases

  betta = 0.01
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + betta *(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.150452
Training accuracy: 7.7%
Validation accuracy: 9.3%
Loss at step 100: 11.701612
Training accuracy: 74.1%
Validation accuracy: 72.1%
Loss at step 200: 4.448113
Training accuracy: 79.5%
Validation accuracy: 76.5%
Loss at step 300: 1.971733
Training accuracy: 82.7%
Validation accuracy: 79.2%
Loss at step 400: 1.128544
Training accuracy: 84.1%
Validation accuracy: 80.9%
Loss at step 500: 0.838712
Training accuracy: 84.7%
Validation accuracy: 81.4%
Loss at step 600: 0.737998
Training accuracy: 84.9%
Validation accuracy: 81.8%
Loss at step 700: 0.702705
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 800: 0.690261
Training accuracy: 84.8%
Validation accuracy: 82.0%
Test accuracy: 88.8%


Logistic model with batch gradient descent without regularisation gives us 82.5% test accuracy. Using regularisaton with some values of betta, we got the best result 88.7% using betta = 0.01.

Now let's try to use regularisation for logistic model with stochastic gradient descent. 

In [22]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(tf_train_dataset, weights) + biases

  betta = 0.01
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + betta *(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)))
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.220215
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 500: 0.749279
Minibatch accuracy: 89.8%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.790895
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.587876
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.687259
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.723446
Minibatch accuracy: 80.5%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.866150
Minibatch accuracy: 77.3%
Validation accuracy: 81.6%
Test accuracy: 88.5%


Logistic model with stochastic gradient descent without regularisation gives us 89.8% test accuracy. Using regularisaton with betta = 0.01 we got 88.5%. Thus, we did not improve the result.

## L2 Regularistaion for neural network model.

In [31]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    num_hidden_nodes = 1554
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits = tf.matmul(layer_1, weights2) + biases2
    
    betta = 0.01
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
        logits=logits) + betta *(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2)
                                 + tf.nn.l2_loss(biases2)))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1),weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1),weights2) + biases2)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
	tf.initialize_all_variables().run()
	print("Two layers netowrk, Initialized")
	for step in range(num_steps):
		offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
		batch_data = train_dataset[offset:(offset + batch_size), :]
		batch_labels = train_labels[offset:(offset + batch_size), :]
		feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
		_, l, predictions = session.run(
			[optimizer, loss, train_prediction], feed_dict=feed_dict)
		if (step % 500 == 0):
			print("Minibatch loss at step %d: %f" % (step, l))
			print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
			print("Validation accuracy: %.1f%%" % accuracy(
				valid_prediction.eval(), valid_labels))
	print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Two layers netowrk, Initialized
Minibatch loss at step 0: 5202.969727
Minibatch accuracy: 5.5%
Validation accuracy: 26.5%
Minibatch loss at step 500: 31.863476
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 1.006653
Minibatch accuracy: 82.8%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.601501
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.621488
Minibatch accuracy: 89.1%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.663468
Minibatch accuracy: 84.4%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 0.832558
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Test accuracy: 90.3%


We've improve the result by 0.5 % (we had accuracy 89.7% without regularisation)